### Tm calculations for polymerases with a DNA binding domain such as Phusion

The Phusion DNA polymerase is a processive version of the Pfu DNA polymerase. 
The enzyme was engineered with a DNA binding domain that tends to stabilize DNA, so
a higher annealing temperature is needed compared to other polymerases such as Taq.

Phusion was first developed and marketed by Finnzyme, and they provided a Tm calculator applet that 
is no longer available.

This applet was written in Java and I decompiled the code and translated into python as the pydna.tm.tmbresluc.
I unfortunately do not have the source code anymore, but I remember verifying that results were identical.

The Finnzyme calculator (and the pydna.tm.tmbresluc function) give a Tm that is quite high.
Phusion is now marketed by Thermofisher Scientific (TFS) and NEB and each provide their 
own Tm calculator:

[Thermofisher Tm](https://www.thermofisher.com/pt/en/home/brands/thermo-scientific/molecular-biology/molecular-biology-learning-center/molecular-biology-resource-library/thermo-scientific-web-tools/tm-calculator.html)

[NEB Tm](https://tmcalculator.neb.com)

These calculators give different values for the same primers and the same settings which are also different from
pydna.tm.tmbresluc and presumably the Finnzyme calculator.

These five primers are used as example for the NEB Tm calculator:

| Name  | Sequence                 | TFS  | NEB | pydna tmbresluc |
|-------|--------------------------|------|-----|-------------|
| P1fwd | AGCGGATAACAATTTCACACAGGA | 65.7 | 62  | 64.3        |
| P1rev | GTAAAACGACGGCCAGT        | 60.2 | 57  | 55.1        |
| pBeta | AGCGGATAACAATTTCAC       | 55.9 | 52  | 51.6        |
| P3fwd | AGCGGATAAGGGCAATTTCAC    | 64.1 | 61  | 62.9        |
| P3rev | GTAAAACGACGGCCA          | 56.4 | 53  | 52.4        |

[TFS Calculator result](Tm-Calculator_Thermo-Fisher-Scientific-PT.pdf)

[NEB Calculator result](NEB-Tm-Calculator.pdf)

In [6]:
from pydna.tm import tm_dbd, tmbresluc

primers = """\
AGCGGATAACAATTTCACACAGGA
GTAAAACGACGGCCAGT
AGCGGATAACAATTTCAC
AGCGGATAAGGGCAATTTCAC
GTAAAACGACGGCCA
""".splitlines()

for p in primers:
    print(tm_dbd(p))

59.62435529786579
53.76249714741526
49.23347605632004
57.929629988766564
49.69451074604375


In [ ]:
from Bio.SeqUtils import MeltingTemp as mt

Biopython offers a [wide range of Tm formulas](https://biopython.org/DIST/docs/api/Bio.SeqUtils.MeltingTemp-module.html) and I will try to recreate the results of the TFS and NEB calculators using Biopython.

Unfortunatley, the descriptions provided by the companies is not enough to positively know how the 
calculations were done. For this reason I will loop through the most important settings provided by Biopython
to see if there is a match or at least a close match.

The primer concentrations were set to 500 nM and the Mg2+ to 1.5 mM as provided by the HF buffer ([description](MAN0013369_Phusion_HF_Buffer_Pack_UG.pdf)). 


There are seven different salt correction formulas 

       1. 16.6 x log[Na+]
          (Schildkraut & Lifson (1965), Biopolymers 3: 195-208)
       2. 16.6 x log([Na+]/(1.0 + 0.7*[Na+]))
          (Wetmur (1991), Crit Rev Biochem Mol Biol 126: 227-259)
       3. 12.5 x log(Na+]
          (SantaLucia et al. (1996), Biochemistry 35: 3555-3562
       4. 11.7 x log[Na+]
          (SantaLucia (1998), Proc Natl Acad Sci USA 95: 1460-1465
       5. Correction for deltaS: 0.368 x (N-1) x ln[Na+]
          (SantaLucia (1998), Proc Natl Acad Sci USA 95: 1460-1465)
       6. (4.29(%GC)-3.95)x1e-5 x ln[Na+] + 9.40e-6 x ln[Na+]^2
          (Owczarzy et al. (2004), Biochemistry 43: 3537-3554)
       7. Complex formula with decision tree and 7 empirical constants.
          Mg2+ is corrected for dNTPs binding (if present)
          (Owczarzy et al. (2008), Biochemistry 47: 5336-5353)


and four different tables with nearest neighbor thermodynamic data.

        - DNA_NN1: values from Breslauer et al. (1986)
        - DNA_NN2: values from Sugimoto et al. (1996)
        - DNA_NN3: values from Allawi & SantaLucia (1997) (default)
        - DNA_NN4: values from SantaLucia & Hicks (2004)

The dnac1 and 2 were set to 500 nM and Mg to 1.5 mM

The primer sequence is the P1fwd (see above).

In [ ]:
from itertools import cycle

tables = cycle(['DNA_NN1', 'DNA_NN2', 'DNA_NN3', 'DNA_NN4'])

tms=[]
for saltcorr in [1,2,3,4,5,6,7]:
    print("saltcorrection", saltcorr)
    for nn_table in [mt.DNA_NN1,mt.DNA_NN2,mt.DNA_NN3,mt.DNA_NN4]:
        print( next(tables),   mt.Tm_NN( "AGCGGATAACAATTTCACACAGGA", # P1fwd
                               nn_table=nn_table,
                               Na=50,
                               Tris=75.0,
                               Mg=1.5, 
                               dnac1=500, 
                               dnac2=500, 
                               dNTPs=0.8, 
                               saltcorr=saltcorr))
    print()

The third value above is 62.12795214651103 (saltcorrection 1 & DNA_NN3) is the closest to 62 which is the value for this primer by NEB Tm.
In the cell below, this setting is tested for all the previously listed primers. 

In [ ]:
primers = """\
P1fwd	 AGCGGATAACAATTTCACACAGGA	
P1rev	 GTAAAACGACGGCCAGT	
pBeta	 AGCGGATAACAATTTCAC	
P3fwd	 AGCGGATAAGGGCAATTTCAC	
P3rev	 GTAAAACGACGGCCA"""

for name, primer in [s.split() for s in primers.splitlines()]:
    tm = mt.Tm_NN( primer,
                   nn_table=mt.DNA_NN3,
                   Na=50,
                   Tris=75.0,
                   Mg=1.5, 
                   dnac1=500, 
                   dnac2=500, 
                   dNTPs=0.8, 
                   saltcorr=1)
    print(name, f"{tm:.0f}")

The output is identical to the one given by NEB. Saltcorrection = 1 corresponds to the one by Schildkraut & Lifson that [NEB says that they use](NEB_help.pdf) on their help page.
The Ta algorithm shoud then be described by the function below.

In [ ]:
def Tm_NEB(primer):
    tm = mt.Tm_NN( primer,
               nn_table=mt.DNA_NN3,
               Na=50,
               Tris=75.0,
               Mg=1.5, 
               dnac1=500, 
               dnac2=500, 
               dNTPs=0.8, 
               saltcorr=1)
    return tm

In [ ]:
def Ta_NEB(fp,rp):
    tms = []
    for primer in [fp,rp]:
        tms.append(Tm_NEB(primer))
    tm = min(tms) + 3.0
    return tm if tm<72 else 72

round(Ta_NEB("AGCGGATAACAATTTCACACAGGA","GTAAAACGACGGCCAGT")) # P1fwd + P1rev

The TFS calculator value for P1fwd AGCGGATAACAATTTCACACAGGA (65.7) corresponds best to the value for saltcorrection 7 & DNA_NN4 = 65.74120743229503 but also for saltcorrection 4 & DNA_NN3= 65.68571994829779. The first case gives:

In [ ]:
primers = """\
P1fwd	 AGCGGATAACAATTTCACACAGGA	
P1rev	 GTAAAACGACGGCCAGT	
pBeta	 AGCGGATAACAATTTCAC	
P3fwd	 AGCGGATAAGGGCAATTTCAC	
P3rev	 GTAAAACGACGGCCA"""

for name, primer in [s.split() for s in primers.splitlines()]:
    tm = mt.Tm_NN( primer,
                   nn_table=mt.DNA_NN4,
                   Na=50,
                   Tris=75.0,
                   Mg=1.5,
                   dnac1=500, 
                   dnac2=500, 
                   dNTPs=0.8, 
                   saltcorr=7)
    print(name, f"{tm:.1f}")

The second case gives:

In [ ]:
primers = """\
P1fwd	 AGCGGATAACAATTTCACACAGGA	
P1rev	 GTAAAACGACGGCCAGT	
pBeta	 AGCGGATAACAATTTCAC	
P3fwd	 AGCGGATAAGGGCAATTTCAC	
P3rev	 GTAAAACGACGGCCA"""

for name, primer in [s.split() for s in primers.splitlines()]:
    tm = mt.Tm_NN( primer,
                   nn_table=mt.DNA_NN3,
                   Na=50,
                   Tris=75.0,
                   Mg=1.5,
                   dnac1=500, 
                   dnac2=500, 
                   dNTPs=0.8, 
                   saltcorr=4)
    print(name, f"{tm:.1f}")

The second case above seems to be a better match. Saltcorrection 4 is the one described by SantaLucia (1998). ThermoFisher makes no reference to this paper afaik.
The code below generate a number of random primers for both calculators in order to test a wider selection of primers.

In [ ]:
# import random
# def randomDNA(size=20, gc=0.5):
#     weights = [gc/2,gc/2,(1-gc)/2,(1-gc)/2]
#     return "".join( random.choices(("G","C","A","T"),weights=weights, k=size) )

# primers = []
# tms = []

# for i in range(100):
#     fp = randomDNA(18)
#     rp = randomDNA(18)
#     primers.append((fp,rp))

# for i,(fp,rp) in enumerate(primers):    
#     print(f"fp{i}", ";",fp,";",f"rp{i}",";",rp)

In [ ]:
import pandas as pd

In [ ]:
NEB = pd.read_csv("tmcalc_batch.txt", names=("pr","seq","Tm","Ta", "comment"), sep="\t")
NEB

In [ ]:
primers = [p.strip() for p in NEB["seq"] ]

In [ ]:
primers

In [ ]:
primers[199]

In [ ]:
# https://www.geeksforgeeks.org/adding-new-column-to-existing-dataframe-in-pandas
NEB["Tm bp"] = [Tm_NEB(p) for p in primers]

In [ ]:
NEB

In [ ]:
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
from itertools import zip_longest
primer_pairs = list( zip_longest(*[iter(primers)]*2) )

In [ ]:
tas = [Ta_NEB(f,r) for f,r in primer_pairs]

In [ ]:
tas_twice = []
for ta in tas:
    tas_twice.append(ta)
    tas_twice.append(ta)

In [ ]:
NEB["Ta bp"] = tas_twice

In [ ]:
NEB

In [ ]:
# https://stackoverflow.com/questions/17812978/how-to-plot-two-columns-of-a-pandas-data-frame-using-points
NEB.plot(x='Tm', y='Tm bp', style='.')

In [ ]:
NEB.plot(x='Ta', y='Ta bp', style='.')